## 1. Load Non-Binary Data

## a) Public Holidays

[OpenHolidays API](https://www.openholidaysapi.org/en/#school-holidays)

In [2]:
# We loaded the school holidays and public holidays data from the OpenHolidays API

import requests
import pandas as pd

# API endpoint for public holidays

api_url_p = "https://openholidaysapi.org/PublicHolidays"

params = {
    "countryIsoCode": "CH",
    "validFrom": "2021-08-28",
    "validTo": "2023-11-08"
}

headers = {"accept": "text/json"}

# Change json to pandas dataframe

response_p = requests.get(api_url_p, params=params, headers=headers)
holidays_data_p = response_p.json()
holidays_df_p = pd.DataFrame(holidays_data_p)

# Extract text

extract_text = lambda col: col[0]["text"] if col else None
holidays_df_p["Name"] = holidays_df_p["name"].apply(extract_text)

# startDate to datetime

holidays_df_p["startDate"] = pd.to_datetime(holidays_df_p["startDate"], format="%Y-%m-%d")
holidays_df_p["endDate"] = pd.to_datetime(holidays_df_p["endDate"], format="%Y-%m-%d")

# Extract subdivision_short_names

holidays_df_p["subdivision_short_names"] = holidays_df_p["subdivisions"].apply(lambda x: [entry["shortName"] for entry in x] if isinstance(x, list) else [])

all_unique_short_names = set()
_ = holidays_df_p["subdivision_short_names"].apply(lambda x: all_unique_short_names.update(x))

# Create columns for each short_name

for short_name in all_unique_short_names:
    holidays_df_p[short_name] = holidays_df_p["subdivision_short_names"].apply(lambda x: short_name in x)

# Drop unnecessary columns

holidays_df_p.drop(columns=["id", "name", "subdivisions", "subdivision_short_names"], axis=1, inplace=True)


# Loop through canton_codes list

canton_codes = ["ZH", "AG", "SZ", "ZG", "SG", "TG"]

for canton_code in canton_codes:
    
    # Check the public holidays for the current canton
    
    canton_public = holidays_df_p.loc[holidays_df_p[canton_code] == True]
    canton_public = canton_public[["startDate", "Name"]]
    canton_public.set_index("startDate", inplace=True)
    canton_public.rename_axis("Date", inplace=True)
    canton_public.reset_index(inplace=True)
    canton_public["Date"] = pd.to_datetime(canton_public["Date"])

    # Import accurate timeframe
    
    timeframe = pd.read_csv("accurate_timeframe.csv")
    timeframe["datetime"] = pd.to_datetime(timeframe["datetime"])

    # Merge timeframe with holidays
    
    timeframe["date"] = timeframe["datetime"].dt.date
    canton_public["date"] = canton_public["Date"].dt.date
    timeframe["date"] = pd.to_datetime(timeframe["date"])
    canton_public["date"] = pd.to_datetime(canton_public["date"])

    merged_df = pd.merge_asof(timeframe, canton_public, left_on=["date"], right_on=["date"], direction="backward", suffixes=("_left", "_right"))

    # Filter the merged dataframe to keep only the accurate dates
    
    merged_df = merged_df[merged_df["date"] == merged_df["Date"]]
    merged_df = merged_df[["datetime", "Name"]]
    merged_df.set_index("datetime", inplace=True)
    merged_df.rename_axis("Date", inplace=True)
    merged_df.rename(columns={"Name": f"Public Holiday {canton_code}"}, inplace=True)

    # Save the result to a CSV file
    
    merged_df.to_csv(f"data/{canton_code.lower()}_public.csv")

    # Display and print the result
    
    display(merged_df)
    
    print(merged_df.shape)


,Public Holiday ZH
Date,
2021-12-26 00:00:00,Stephanstag
2021-12-26 01:00:00,Stephanstag
2021-12-26 02:00:00,Stephanstag
2021-12-26 03:00:00,Stephanstag
2021-12-26 04:00:00,Stephanstag
...,...
2023-09-17 19:00:00,"Eidgenössischer Dank-, Buss- und Bettag"
2023-09-17 20:00:00,"Eidgenössischer Dank-, Buss- und Bettag"
2023-09-17 21:00:00,"Eidgenössischer Dank-, Buss- und Bettag"


(432, 1)


,Public Holiday AG
Date,
2021-11-01 00:00:00,Allerheiligen
2021-11-01 01:00:00,Allerheiligen
2021-11-01 02:00:00,Allerheiligen
2021-11-01 03:00:00,Allerheiligen
2021-11-01 04:00:00,Allerheiligen
...,...
2023-11-01 19:00:00,Allerheiligen
2023-11-01 20:00:00,Allerheiligen
2023-11-01 21:00:00,Allerheiligen


(552, 1)


,Public Holiday SZ
Date,
2021-11-01 00:00:00,Allerheiligen
2021-11-01 01:00:00,Allerheiligen
2021-11-01 02:00:00,Allerheiligen
2021-11-01 03:00:00,Allerheiligen
2021-11-01 04:00:00,Allerheiligen
...,...
2023-11-01 19:00:00,Allerheiligen
2023-11-01 20:00:00,Allerheiligen
2023-11-01 21:00:00,Allerheiligen


...

(552, 1)


,Public Holiday SG
Date,
2021-11-01 00:00:00,Allerheiligen
2021-11-01 01:00:00,Allerheiligen
2021-11-01 02:00:00,Allerheiligen
2021-11-01 03:00:00,Allerheiligen
2021-11-01 04:00:00,Allerheiligen
...,...
2023-11-01 19:00:00,Allerheiligen
2023-11-01 20:00:00,Allerheiligen
2023-11-01 21:00:00,Allerheiligen


(360, 1)


,Public Holiday TG
Date,
2021-12-26 00:00:00,Stephanstag
2021-12-26 01:00:00,Stephanstag
2021-12-26 02:00:00,Stephanstag
2021-12-26 03:00:00,Stephanstag
2021-12-26 04:00:00,Stephanstag
...,...
2023-09-17 19:00:00,"Eidgenössischer Dank-, Buss- und Bettag"
2023-09-17 20:00:00,"Eidgenössischer Dank-, Buss- und Bettag"
2023-09-17 21:00:00,"Eidgenössischer Dank-, Buss- und Bettag"


(336, 1)


## b) School Holidays

[OpenHolidays API](https://www.openholidaysapi.org/en/#school-holidays)

In [3]:
# Again the same for school holidays

from pandas.tseries.offsets import DateOffset

# API endpoint for school holidays

api_url_s = "https://openholidaysapi.org/SchoolHolidays"

params = {
    "countryIsoCode": "CH",
    "validFrom": "2021-08-28",
    "validTo": "2023-11-08"
}

headers = {'accept': 'text/json'}

# Change json to pandas dataframe

response_s = requests.get(api_url_s, params=params, headers=headers)
holidays_data_s = response_s.json()
holidays_df_s = pd.DataFrame(holidays_data_s)

# Extract text

extract_text = lambda col: col[0]["text"] if col else None
holidays_df_s["Name"] = holidays_df_s["name"].apply(extract_text)

# startDate to datetime

holidays_df_s["startDate"] = pd.to_datetime(holidays_df_s["startDate"], format="%Y-%m-%d")
holidays_df_s["endDate"] = pd.to_datetime(holidays_df_s["endDate"], format="%Y-%m-%d")

# Extract subdivision_short_names

holidays_df_s["subdivision_short_names"] = holidays_df_s["subdivisions"].apply(lambda x: [entry["shortName"] for entry in x] if isinstance(x, list) else [])
all_unique_short_names_s = set()
_ = holidays_df_s["subdivision_short_names"].apply(lambda x: all_unique_short_names_s.update(x))

# Create columns for each short_name

for short_name in all_unique_short_names_s:
    holidays_df_s[short_name] = holidays_df_s["subdivision_short_names"].apply(lambda x: short_name in x)

# Drop unnecessary columns

holidays_df_s.drop(columns=["id", "name", "subdivisions", "subdivision_short_names", "comment"], axis=1, inplace=True)

# Loop through canton_codes list

for canton_code in canton_codes:
    
    # Check the school holidays for the current canton
    
    canton_school = holidays_df_s.loc[holidays_df_s[canton_code] == True]
    canton_school = canton_school[["startDate", "endDate", "Name"]]

    canton_school.set_index("startDate", inplace=True)
    canton_school.rename_axis("Date", inplace=True)
    canton_school.reset_index(inplace=True)
    canton_school["Date"] = pd.to_datetime(canton_school["Date"])

    # Import accurate timeframe
    
    timeframe = pd.read_csv("accurate_timeframe.csv")
    timeframe["datetime"] = pd.to_datetime(timeframe["datetime"])

    # Merge timeframe with school holidays
    
    timeframe["date"] = timeframe["datetime"].dt.date
    canton_school["date"] = canton_school["Date"].dt.date
    timeframe["date"] = pd.to_datetime(timeframe["date"])
    canton_school["date"] = pd.to_datetime(canton_school["date"])

    merged_df = pd.merge_asof(timeframe, canton_school, on="date", direction="backward", suffixes=("_left", "_right"))

    # Filter the merged DataFrame to keep only the accurate dates
    
    merged_df = merged_df[merged_df["datetime"] < merged_df["endDate"] + DateOffset(days=1)]
    merged_df = merged_df[["datetime", "Name"]]
    merged_df.set_index("datetime", inplace=True)
    merged_df.rename_axis("Date", inplace=True)
    merged_df.rename(columns={"Name": f"School Holidays {canton_code}"}, inplace=True)

    # Save the result to a CSV file
    
    merged_df.to_csv(f"data/{canton_code.lower()}_school_holidays.csv")

    # Display and print the result
    
    display(merged_df)
    print(merged_df.shape)


,School Holidays ZH
Date,
2021-10-09 00:00:00,Herbst
2021-10-09 01:00:00,Herbst
2021-10-09 02:00:00,Herbst
2021-10-09 03:00:00,Herbst
2021-10-09 04:00:00,Herbst
...,...
2023-10-22 19:00:00,Herbst
2023-10-22 20:00:00,Herbst
2023-10-22 21:00:00,Herbst


(4392, 1)


,School Holidays AG
Date,
2021-10-02 00:00:00,Herbst
2021-10-02 01:00:00,Herbst
2021-10-02 02:00:00,Herbst
2021-10-02 03:00:00,Herbst
2021-10-02 04:00:00,Herbst
...,...
2023-10-15 19:00:00,Herbst
2023-10-15 20:00:00,Herbst
2023-10-15 21:00:00,Herbst


(3744, 1)


,School Holidays SZ
Date,
2021-10-02 00:00:00,Herbst
2021-10-02 01:00:00,Herbst
2021-10-02 02:00:00,Herbst
2021-10-02 03:00:00,Herbst
2021-10-02 04:00:00,Herbst
...,...
2023-10-15 19:00:00,Herbst
2023-10-15 20:00:00,Herbst
2023-10-15 21:00:00,Herbst


...

(5400, 1)


,School Holidays SG
Date,
2021-10-02 00:00:00,Herbst
2021-10-02 01:00:00,Herbst
2021-10-02 02:00:00,Herbst
2021-10-02 03:00:00,Herbst
2021-10-02 04:00:00,Herbst
...,...
2023-10-22 19:00:00,Herbst
2023-10-22 20:00:00,Herbst
2023-10-22 21:00:00,Herbst


(4896, 1)


,School Holidays TG
Date,
2021-10-09 00:00:00,Herbst
2021-10-09 01:00:00,Herbst
2021-10-09 02:00:00,Herbst
2021-10-09 03:00:00,Herbst
2021-10-09 04:00:00,Herbst
...,...
2023-10-22 19:00:00,Herbst
2023-10-22 20:00:00,Herbst
2023-10-22 21:00:00,Herbst


(4847, 1)


## c) Closed stores

This dataset was created manually by taking the individual holiday dates from the official Zurich government website for the years 2021, 2022 & 2023 (Canton of Zurich, 2023)

In [4]:
# Importing the time frame we had set 

df_accurate_timeframe = pd.read_csv("accurate_timeframe.csv")

# Importing the excel csv with the dates of when the stores are all closed

df_closing_holidays = pd.read_excel("feiertage ladenschliessung.xlsx")
df_closing_holidays = df_closing_holidays.loc[df_closing_holidays["Feiertag"] != "Stephanstag"]
df_closing_holidays

# Setting the datetime column of the accurate time frame to datetime 

df_accurate_timeframe["datetime"]= pd.to_datetime(df_accurate_timeframe["datetime"])

# Setting the Datum column to datetime and renaming it to datetime, so it'll match the accurate time frame

df_closing_holidays["datetime"]= pd.to_datetime(df_closing_holidays["Datum"])

# Making sure it's of type datetime

df_closing_holidays.info()

# Function that resamples the daily time series data to an hourly

hourly_standard = pd.DataFrame()

for _, row in df_closing_holidays.iterrows():
    date_range = pd.date_range(row['Datum'], periods=24, freq='H')
    temp_df = pd.DataFrame({'Feiertag': [row['Feiertag']] * 24, 'Datum': date_range})
    hourly_standard = pd.concat([hourly_standard, temp_df])

# Reset the index of the new DataFrame

hourly_standard.reset_index(drop=True, inplace=True)

# Assigning a 1.0 to the dates where the stores are closed

hourly_standard["stores_closed_on_that_day"] = 1.0

# Dropping the column Feiertage

hourly_closed_holidays = hourly_standard.drop("Feiertag", axis = 1)

# Renaming the Datum column to datetime

hourly_closed_holidays.rename(columns = {"Datum": "datetime"}, inplace = True)

# Making sure that there are no duplicates 

hourly_closed_holidays.duplicated().sum()

# Merging the df containing the dates of the closed stores with the accurate time frame

hourly_closed_holidays_final = pd.merge(df_accurate_timeframe, hourly_closed_holidays, how ="left", on="datetime")

# Filling all the dates where the stores are open with 0.0

hourly_closed_holidays_final = hourly_closed_holidays_final.fillna(0.0)

# Saving the dt to a csv

hourly_closed_holidays_final.to_csv("data/stores_closed.csv", index=False)
hourly_closed_holidays_final.tail()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16 entries, 0 to 17
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Feiertag  16 non-null     object        
 1   Datum     16 non-null     datetime64[ns]
 2   datetime  16 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(1)
memory usage: 512.0+ bytes


,datetime,stores_closed_on_that_day
18378,2023-11-07 20:00:00,0.0
18379,2023-11-07 21:00:00,0.0
18380,2023-11-07 22:00:00,0.0
18381,2023-11-07 23:00:00,0.0
18382,2023-11-08 00:00:00,0.0


## d) Influential Events

This dataset was also constructed manually by researching the dates of each big event that take place at or around the Bahnhofstrasse. 

In [6]:
# Importing the excel csv with the dates of big events

df_grossevents = pd.read_excel("grossevents zürich.xlsx")

# Setting the Datum column to datetime and renaming it to datetime, so it'll match the accurate time frame

df_grossevents["datetime"]= pd.to_datetime(df_grossevents["Datum"])

# Function that resamples the daily time series data to an hourly

hourly_standard_g_e = pd.DataFrame()

for _, row in df_grossevents.iterrows():
    date_range = pd.date_range(row['datetime'], periods=24, freq='H')
    temp_df = pd.DataFrame({'Event': [row['Event']] * 24, 'datetime': date_range})
    hourly_standard_g_e = pd.concat([hourly_standard_g_e, temp_df])

# Reset the index of the new DataFrame

hourly_standard_g_e.reset_index(drop=True, inplace=True)

# Assigning a 1.0 to the dates when big events are happening

hourly_standard_g_e["Event"] = 1.0

# Merging the df containing the dates of the big events with the accurate time frame

hourly_gross_events_final = pd.merge(df_accurate_timeframe, hourly_standard_g_e, how ="outer", on="datetime")

# Filling all the dates where no big events are happening with 0.0

hourly_gross_events_final = hourly_gross_events_final.fillna(0.0)

# Saving the dt to a csv

hourly_gross_events_final.to_csv("data/influential_events.csv", index=False)

## 2. Load Non-Binary Hourly Data

## a) Weather Data

[Weather data is part of the target variable data set](https://data.stadt-zuerich.ch/dataset/hystreet_fussgaengerfrequenzen)

In [7]:
# Import the dataset

import pandas as pd
import numpy as np

url_cs = "https://data.stadt-zuerich.ch/dataset/hystreet_fussgaengerfrequenzen/download/hystreet_fussgaengerfrequenzen_seit2021.csv"

BHF = pd.read_csv(
    url_cs,
    sep=",",
    encoding="utf-8",
)

# Convert to datetime

BHF.timestamp = pd.to_datetime(BHF.timestamp, format="%Y-%m-%dT%XZ")


# Only keep Weather Data and create Dummies

weather = BHF[["timestamp", "location_id", "weather_condition", "temperature"]]

weather = pd.get_dummies(weather, columns=["weather_condition"], drop_first=True)
weather.columns = weather.columns.str.replace("weather_condition_", "")

In [8]:
# Keep binary weather data

weather = weather[["timestamp", "clear-night", "cloudy", "fog", "partly-cloudy-day", "partly-cloudy-night", "rain", "snow", "wind"]]

# Filter to end-date

end_date = "2023-11-08"
weather = weather.loc[weather["timestamp"] <= end_date]
weather.set_index("timestamp", inplace = True)
weather.rename_axis("Date", inplace = True)
weather.reset_index(inplace = True)

# Shift values to avoid data leakage

weather = weather.shift()
weather.to_csv("data/weather.csv", index = False)
display(weather)


,clear-night,cloudy,fog,partly-cloudy-day,partly-cloudy-night,rain,snow,wind
Date,,,,,,,,
2021-09-28 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-09-28 22:00:00,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-09-28 22:00:00,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-09-28 23:00:00,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-09-28 23:00:00,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
2023-11-07 23:00:00,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2023-11-07 23:00:00,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2023-11-08 00:00:00,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
